In [2]:
from SPARQLWrapper import SPARQLWrapper, JSON
import time

import sys
from pathlib import Path

repo_root = Path.cwd().resolve()
src_path = repo_root / "src"
if src_path.exists():
    sys.path.insert(0, str(src_path))
    
from gebietsstammdaten_zh import (
    api_health,
    bezirk_mit_gemeinden,
    gemeinde_suchen,
    read_csv_from_url,
    build_gemeinde_historisierung,
    write_csv_rows,
)

ALLOW_NETWORK = True

In [9]:
# --- 1️⃣ SPARQL-Abfrage für Zürcher Gemeinden ---
sparql_query = """
SELECT * WHERE {
  ?gemeinde wdt:P31 wd:Q70208.
  {
    ?gemeinde wdt:P131 wd:Q11943.
  }
  UNION
  {
    ?gemeinde wdt:P131 ?bezirk.
    ?bezirk wdt:P131 wd:Q11943.
  }
  ?gemeinde wdt:P1448 ?officialName.
  ?gemeinde wdt:P771 ?municipalityCode.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "de". }
}
ORDER BY ?officialName
"""

sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
sparql.setQuery(sparql_query)
sparql.setReturnFormat(JSON)

results = sparql.query().convert()
rows = results["results"]["bindings"]

In [5]:
# --- 2️⃣ Durch alle Gemeinden iterieren und API abfragen ---
for row in rows:
    qid = row["gemeinde"]["value"].split("/")[-1]
    official_name = row["officialName"]["value"]
    bezirk_label = row["bezirkLabel"]["value"]

    # API-Abfrage mit helper-Funktion
    api_treffer = gemeinde_suchen(official_name, dry_run=False)

    time.sleep(0.3)  # API schonen

    if api_treffer:
        api_gemeinde_name = api_treffer[0]["gemeinde"]["gemeinde_name"]
        api_gemeinde_code = api_treffer[0]["gemeinde"]["gemeinde_code"]
        api_bezirk_name = api_treffer[0]["bezirk"]["bezirk_name"]
        api_bezirk_code = api_treffer[0]["bezirk"]["bezirk_code"]

        # Vergleich
        match_gemeinde = official_name == api_gemeinde_name

        if not match_gemeinde:


            
        match_bezirk = bezirk_label == api_bezirk_name

        print(f"{official_name} ({qid}):")
        print(f"  API Gemeinde: {api_gemeinde_name} | Code: {api_gemeinde_code}")
        print(f"  API Bezirk: {api_bezirk_name} | Code: {api_bezirk_code}")
        print(f"  Match Gemeinde: {match_gemeinde}, Match Bezirk: {match_bezirk}")
        print("-" * 60)

    else:
        print(f"{official_name} ({qid}): ✘ Kein API-Treffer")


gemeindename=K%C3%BCsnacht
Küsnacht (Q69216):
  API Gemeinde: Küsnacht (ZH) | Code: 154
  API Bezirk: Meilen | Code: 107
  Match Gemeinde: False, Match Bezirk: False
------------------------------------------------------------
gemeindename=Adliswil
Adliswil (Q68210):
  API Gemeinde: Adliswil | Code: 131
  API Bezirk: Horgen | Code: 106
  Match Gemeinde: True, Match Bezirk: False
------------------------------------------------------------
gemeindename=Aesch+%28ZH%29
Aesch (ZH) (Q68342):
  API Gemeinde: Aesch (ZH) | Code: 241
  API Bezirk: Dietikon | Code: 111
  Match Gemeinde: True, Match Bezirk: False
------------------------------------------------------------
gemeindename=Aeugst+am+Albis
Aeugst am Albis (Q64945):
  API Gemeinde: Aeugst am Albis | Code: 1
  API Bezirk: Affoltern | Code: 101
  Match Gemeinde: True, Match Bezirk: False
------------------------------------------------------------
gemeindename=Affoltern+am+Albis
Affoltern am Albis (Q68290):
  API Gemeinde: Affoltern am A